In [1]:
import json
import lzma
import matplotlib.pyplot as plt
import nltk
import numpy as np
import os
import pandas as pd
import seaborn as sns
from collections import Counter
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from pandas.io.json import json_normalize
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/11400213/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
from bs4 import BeautifulSoup
import lxml

In [3]:
base_path = "./data/xml"
state='north_carolina.xz'
f = lzma.open(os.path.join(base_path,state),"rb")
state_data = f.readlines()
f.close()

In [4]:
data_json = [json.loads(line) for line in state_data]
print(f'Flattening data for {state}')
data = json_normalize(data_json)

Flattening data for north_carolina.xz


In [5]:
data['decision_date_p'] = pd.to_datetime(data.decision_date,errors='coerce')
data['decision_year'] = data.decision_date_p.dt.year

In [6]:
print(data.shape)
print(data.columns)

def get_text(type,casebody):
 rtext = ''
 for txt in casebody.find_all(type):
    rtext = rtext + ' ' + txt.text
 return rtext



(97600, 33)
Index(['id', 'url', 'name', 'name_abbreviation', 'decision_date',
       'docket_number', 'first_page', 'last_page', 'citations', 'cites_to',
       'frontend_url', 'preview', 'volume.volume_number', 'volume.barcode',
       'volume.url', 'reporter.url', 'reporter.full_name', 'reporter.id',
       'court.name', 'court.id', 'court.name_abbreviation', 'court.url',
       'court.slug', 'jurisdiction.name_long', 'jurisdiction.name',
       'jurisdiction.whitelisted', 'jurisdiction.id', 'jurisdiction.url',
       'jurisdiction.slug', 'casebody.status', 'casebody.data',
       'decision_date_p', 'decision_year'],
      dtype='object')


In [7]:
casebody = pd.DataFrame(columns =  ('headnotes','Opinion', 'casenum'  ), index=range(50000))
j = 1
random_sample =  data.sample(n = 50000, replace = False) 
for i in range(50000):
  markup = random_sample['casebody.data'].iloc[i]
  soup = BeautifulSoup(markup, "xml")
  hn = get_text('headnotes', soup).replace('\n', ' ')
  opinion = get_text('opinion', soup).replace('\n', ' ')
  if(len(hn)> 250 and len(opinion)> len(hn)):
    casebody['headnotes'][i] = hn
    casebody['Opinion'][i] = opinion
    casebody['casenum'][i] = j
    j= j+1
    


In [8]:
print (casebody.shape)
casebody = casebody.dropna()
print (casebody.shape)
casebody.head(3)
casebosy = casebody.reset_index()
casebody = casebody.assign(headnotes_split="",Opinion_split="", extracted_sentences="", selected_sentences="")
casebody.head()



(50000, 3)
(35081, 3)


,headnotes,Opinion,casenum,headnotes_split,Opinion_split,extracted_sentences,selected_sentences
0,"A debtor, and those who by a fraudulent deed ...","Pearson, J. The deed of trust from W. R. to ...",1,,,,
1,Trial § SS— Where the inadvertence of a witne...,Per Curiam. The witnesses appear to have use...,2,,,,
2,Appeal and Error* D a. — Court may disregard ...,"Stacy, O. J. This is the second appeal in th...",3,,,,
3,"If the record of the Court below is false, it...","Rodman, J. This is an application to rehear ...",4,,,,
4,1. Appeal and Error § 322 (NCI4th)— date of n...,"LEWIS, Judge. The issue in this case is whet...",5,,,,


In [22]:
x = len(casebody)
print(x)
for i in range(x):
    
    splitcase = casebody.iloc[i]
    casebody['headnotes_split'][i]= [word_tokenize(t) for t in sent_tokenize(splitcase.headnotes)]
    casebody['Opinion_split'][i] = [word_tokenize(t) for t in sent_tokenize(splitcase.Opinion)]
    

35081


/usr/share/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/share/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [23]:

casebody.head()

,index,headnotes,Opinion,casenum,headnotes_split,Opinion_split,extracted_sentences,selected_sentences
0,0,"A debtor, and those who by a fraudulent deed ...","Pearson, J. The deed of trust from W. R. to ...",1,"[[A, debtor, ,, and, those, who, by, a, fraudu...","[[Pearson, ,, J, .], [The, deed, of, trust, fr...",,
1,1,Trial § SS— Where the inadvertence of a witne...,Per Curiam. The witnesses appear to have use...,2,"[[Trial, §, SS—, Where, the, inadvertence, of,...","[[Per, Curiam, .], [The, witnesses, appear, to...",,
2,2,Appeal and Error* D a. — Court may disregard ...,"Stacy, O. J. This is the second appeal in th...",3,"[[Appeal, and, Error*, D, a, .], [—, Court, ma...","[[Stacy, ,, O., J, .], [This, is, the, second,...",,
3,3,"If the record of the Court below is false, it...","Rodman, J. This is an application to rehear ...",4,"[[If, the, record, of, the, Court, below, is, ...","[[Rodman, ,, J, .], [This, is, an, application...",,
4,4,1. Appeal and Error § 322 (NCI4th)— date of n...,"LEWIS, Judge. The issue in this case is whet...",5,"[[1, .], [Appeal, and, Error, §, 322, (, NCI4t...","[[LEWIS, ,, Judge, .], [The, issue, in, this, ...",,


In [33]:
import re

def _get_ngrams(n, text):
    """Calcualtes n-grams.

    Args:
      n: which n-grams to calculate
      text: An array of tokens

    Returns:
      A set of n-grams
    """
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set


def _get_word_ngrams(n, sentences):
    """Calculates word n-grams for multiple sentences.
    """
    assert len(sentences) > 0
    assert n > 0

    # words = _split_into_words(sentences)

    words = sum(sentences, [])
    # words = [w for w in words if w not in stopwords]
    return _get_ngrams(n, words)


def cal_rouge(evaluated_ngrams, reference_ngrams):
    reference_count = len(reference_ngrams)
    evaluated_count = len(evaluated_ngrams)

    overlapping_ngrams = evaluated_ngrams.intersection(reference_ngrams)
    overlapping_count = len(overlapping_ngrams)

    if evaluated_count == 0:
        precision = 0.0
    else:
        precision = overlapping_count / evaluated_count

    if reference_count == 0:
        recall = 0.0
    else:
        recall = overlapping_count / reference_count

    f1_score = 2.0 * ((precision * recall) / (precision + recall + 1e-8))
    return {"f": f1_score, "p": precision, "r": recall}


def greedy_selection(doc_sent_list, abstract_sent_list, summary_size):
    def _rouge_clean(s):
        return re.sub(r'[^a-zA-Z0-9 ]', '', s)
    max_rouge = 0.0
    abstract = sum(abstract_sent_list, [])
    #abstract = abstract_sent_list
    abstract = _rouge_clean(' '.join(abstract)).split()
    sents = [_rouge_clean(' '.join(s)).split() for s in doc_sent_list]
    evaluated_1grams = [_get_word_ngrams(1, [sent]) for sent in sents]
    #print(evaluated_1grams)
    reference_1grams = _get_word_ngrams(1, [abstract])
    evaluated_2grams = [_get_word_ngrams(2, [sent]) for sent in sents]
    reference_2grams = _get_word_ngrams(2, [abstract])

    selected = []

    for s in range(summary_size):
        cur_max_rouge = max_rouge
        cur_id = -1
        
        for i in range(len(sents)):
            if (i in selected):
                continue
            c = selected + [i]
            candidates_1 = [evaluated_1grams[idx] for idx in c]
            candidates_1 = set.union(*map(set, candidates_1))
            candidates_2 = [evaluated_2grams[idx] for idx in c]
            candidates_2 = set.union(*map(set, candidates_2))
            rouge_1 = cal_rouge(candidates_1, reference_1grams)['f']
            rouge_2 = cal_rouge(candidates_2, reference_2grams)['f']
            rouge_score = rouge_1 + rouge_2
            if rouge_score > cur_max_rouge:
                cur_max_rouge = rouge_score
                cur_id = i
        if (cur_id == -1):
            return sorted(selected)
        selected.append(cur_id)
        max_rouge = cur_max_rouge
       
    
    return sorted(selected)

In [61]:

for i in range(len(casebody)):
      splitcase = casebody.iloc[i] 
      split_opinions = splitcase['Opinion_split']
      split_headnotes = splitcase['headnotes_split']
      hl = len(split_headnotes)
      sentlist = greedy_selection(split_opinions,split_headnotes,hl)
      opinsent = [" ".join('0') for tgt in split_opinions]
      extracted_sentences = [" ".join(split_opinions[tgt]) for tgt in sentlist]
      if i in (1,10,100):
        print('-----------------------------------')
        print('Data for ', i)
        print(splitcase['Opinion'])
        print(splitcase['headnotes'])
        print(extracted_sentences)
        print(splitcase['selected_sentences'])
        print(sentlist)
      for j in sentlist:
        opinsent[j] = '1'
      casebody['extracted_sentences'][i] = extracted_sentences
      casebody['selected_sentences'][i] = opinsent
      
      if i in (1,10,100):
        print(casebody['extracted_sentences'][i])
        print(casebody['selected_sentences'][i])

/usr/share/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/share/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


-----------------------------------
Data for  1
  Per Curiam. The witnesses appear to have used the square foot value as the unit by which to fix the total value. Admittedly the lot was 188.36 ft. x 92 ft. One of the City’s "witnesses, by mistake, had calculated total value on the basis of 90 rather than 92 feet. While he was on the stand his error was discovered and corrected before the jury. However, in the Court’s statement of the contention of the parties and in the recapitulation of the evidence, the Court did not specifically call attention to the fact that the witness Smith, in his direct examination, had based his estimate of value on 90 rather than 92 feet frontage. At the conclusion of the charge, the appellants failed to request the Court to make any correction or amplification, either of the evidence or the contention of the parties. The defendants cite and rely on the cases of Town of Davidson v. Stough, 258 N.C. 23, 127 S.E. 2d 762 as furnishing ground for a new trial. Ho

KeyboardInterrupt: 

In [ ]:

casebody.shape

In [62]:
casebody.head()

,index,headnotes,Opinion,casenum,headnotes_split,Opinion_split,extracted_sentences,selected_sentences
0,0,"A debtor, and those who by a fraudulent deed ...","Pearson, J. The deed of trust from W. R. to ...",1,"[[A, debtor, ,, and, those, who, by, a, fraudu...","[[Pearson, ,, J, .], [The, deed, of, trust, fr...",[The deed of trust from W. R. to James C. Skin...,"[0, 1, 0, 0, 1, 0, 0]"
1,1,Trial § SS— Where the inadvertence of a witne...,Per Curiam. The witnesses appear to have use...,2,"[[Trial, §, SS—, Where, the, inadvertence, of,...","[[Per, Curiam, .], [The, witnesses, appear, to...",[While he was on the stand his error was disco...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,2,Appeal and Error* D a. — Court may disregard ...,"Stacy, O. J. This is the second appeal in th...",3,"[[Appeal, and, Error*, D, a, .], [—, Court, ma...","[[Stacy, ,, O., J, .], [This, is, the, second,...","[When the case was called for trial again , la...","[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0]"
3,3,"If the record of the Court below is false, it...","Rodman, J. This is an application to rehear ...",4,"[[If, the, record, of, the, Court, below, is, ...","[[Rodman, ,, J, .], [This, is, an, application...","[If the record is false in that respect , it c...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
4,4,1. Appeal and Error § 322 (NCI4th)— date of n...,"LEWIS, Judge. The issue in this case is whet...",5,"[[1, .], [Appeal, and, Error, §, 322, (, NCI4t...","[[LEWIS, ,, Judge, .], [The, issue, in, this, ...",[The issue in this case is whether a foreign l...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [64]:
print(casebody.extracted_sentences[30000])
print(casebody.headnotes[30000])
print(casebody.Opinion[30000])
casebody.selected_sentences[30000]

['If he intended to press his attentions to the point of sexual gratification by seduction , if possible , by force if necessary , and with that intent laid his hands upon her with her full consent and approval he would be guilty .', 'G. S. , 14-22 .', 'So to convict , the State must prove ( 1 ) an assault by a male upon a female ( 2 ) with intent to commit rape , and the felonious intent is the intent to gratify his passion on the person of the woman at all events against her will and notwithstanding any resistance she may make .', 'Hence his advances , however immoral , did not constitute an assault until they reached a point they were offensive to her .', 'When there are conflicting instructions to the jury upon a material point , the one correct and the other incorrect , a new trial must be granted .']
 1. Rape § 24— In order for a conviction under G. S., 14-22, there must be an assault by a male upon a female with intent to commit rape, which felonious intent is the intent to grat

['0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '1',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '1',
 '0',
 '0',
 '0',
 '0',
 '0']